In [2]:
import pickle
import os, random
import h5py, copy
import numpy as np
import pandas as pd
from tqdm import tqdm

### Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

### Torchvision
from torchsummary import summary
import torchvision.datasets as dest
import torchvision.transforms as transformers

### Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
### seed_everythin
seed = 1987
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.random.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Parameters

In [5]:
ngpu = 1
batch_size = 512
num_workers = 1
channels = 3
data_range = 10
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print(f'Device: {device}')

Device: cuda:0


### Read data from pickle

In [6]:
filename = 'selected_data.pkl'
outfile = open(filename,'rb')
data = pickle.load(outfile)
xdata = np.array(data['data'], dtype = 'float32')
ylablel = np.array(data['labels'], dtype = 'float32')

### Train test split

In [7]:
### Split data
X_train, X_val, y_train, y_val = train_test_split(xdata, ylablel, test_size=0.2)
X_train.shape, X_val.shape

((145864, 50, 3), (36466, 50, 3))

### Normalize data

In [8]:
xscalers = {}
for i in range(xdata.shape[2]):
    xscalers[i] = StandardScaler()
    X_train[:, :, i] = xscalers[i].fit_transform(X_train[:, :, i])

yscaler = StandardScaler()
y_train = yscaler.fit_transform(y_train.reshape(-1, 1))

In [9]:
scalers = {}
for i in range(xdata.shape[2]):
    X_val[:, :, i] = xscalers[i].transform(X_val[:, :, i])

y_val = yscaler.transform(y_val.reshape(-1, 1))

In [10]:
y_val.max(), y_train.max()

(6.706501, 7.02627)

### Create dataloader

In [11]:
### Dataset
class SeismicDataset(Dataset):
    def __init__(self, method):
        if method == 'train':
            shp = X_train.shape
            self.xdata = torch.Tensor(X_train).view(shp[0], shp[2], shp[1])[:,:, 0:data_range]
            self.ylablel = torch.Tensor(y_train)
        elif method == 'val':
            shp = X_val.shape
            self.xdata = torch.Tensor(X_val).view(shp[0], shp[2], shp[1])[:,:, 0:data_range]
            self.ylablel = torch.Tensor(y_val)
            
        
    def __len__(self):
        return len(self.xdata)
        
    def __getitem__(self, indx):
        return self.xdata[indx], self.ylablel[indx]
        
### Dataloader
train_dataloader = torch.utils.data.DataLoader(
    SeismicDataset('train'),
    batch_size= batch_size,
    shuffle = True,
    num_workers = num_workers
)

val_dataloader = torch.utils.data.DataLoader(
    SeismicDataset('val'),
    batch_size= batch_size,
    shuffle = True,
    num_workers = num_workers
)

In [12]:
next(iter(train_dataloader))[0].size()

torch.Size([512, 3, 10])

### Model

In [13]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv1d(in_channels, out_channels,  bias = False, **kwargs)
        self.batchnorm = nn.BatchNorm1d(out_channels)
        
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [14]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1_pool):
        super(InceptionBlock, self).__init__()
        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size = 1)
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red_3x3, kernel_size = 1),
            ConvBlock(red_3x3, out_3x3, kernel_size = 3, padding = 1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red_5x5, kernel_size = 1),
            ConvBlock(red_5x5, out_5x5, kernel_size = 5, padding = 2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride = 1, padding = 1),
            ConvBlock(in_channels, out_1x1_pool, kernel_size = 1)
        )
        
        
    def forward(self, x):
        return torch.cat(
            [self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1
        )
       

In [15]:
class SeismicNet(nn.Module):
    def __init__(self):
        super(SeismicNet, self).__init__()
        self.conv1 = ConvBlock(3, 192, kernel_size = 2, stride = 2)
        self.inception_1a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception_1b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding = 1)
        
        self.averagepool1 = nn.AvgPool1d(kernel_size= 7, stride= 1)
        self.fc1 = nn.Linear(3360, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout1 = nn.Dropout2d(p = 0.25)
        self.dropout2 = nn.Dropout2d(p = 0.20)
        self.dropout3 = nn.Dropout2d(p = 0.15)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.inception_1a(x)
        x = self.inception_1b(x)
        x = self.maxpool1(x)

        x = self.averagepool1(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        return x

In [16]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.input_dim = channels*data_range
        self.main = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.35),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.35),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
        
    def forward(self, x):
        x = Variable(torch.flatten(x, start_dim = 1))
        x = self.main(x)
        return x

In [17]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.input_dim = channels*data_range
        self.fc1 = nn.Linear(self.input_dim, 128)
        self.relu = nn.ReLU()
        self.dp1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 64)
        self.dp2 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 1)
        
    def forward(self, x):
        x = Variable(torch.flatten(x, start_dim = 1))
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dp1(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dp2(x)
        
        x = self.fc3(x)
        x = self.relu(x)
        
        x = self.fc4(x)
        return x

In [18]:
model = SimpleModel().to(device) #SeismicNet().to(device)
if (device.type == 'cuda' and (ngpu> 1)):
    model = nn.DataParallel(model, list(range(ngpu)))

### Training Model

In [19]:
tb = SummaryWriter()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0003)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=10, factor = 0.15, verbose=True
)

# scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0003, max_lr=0.1, cycle_momentum=False)

In [20]:
def train(epoch):
    model.train()
    losses = []
    for j, (data, label) in enumerate(train_dataloader, 0):
        optimizer.zero_grad()
        data, y = data.to(device), label.to(device)
        output = model(data)
        loss = criterion(output, y)
        loss.backward()
               
        tb.add_histogram('gradients/fc1', model.fc1.weight.grad, epoch)
        tb.add_histogram('gradients/fc2', model.fc2.weight.grad, epoch)
        tb.add_histogram('gradients/fc3', model.fc3.weight.grad, epoch)
        tb.add_histogram('gradients/fc4', model.fc4.weight.grad, epoch)
        
        tb.add_scalars('fc_grad/mean',{
            'fc1': model.fc1.weight.grad.abs().mean(),
            'fc2': model.fc2.weight.grad.abs().mean(),
            'fc3': model.fc3.weight.grad.abs().mean(),
            'fc4': model.fc4.weight.grad.abs().mean()
            
        }, epoch)
        
        tb.add_scalars('fc_grad/variance',{
            'fc1': model.fc1.weight.grad.abs().var(),
            'fc2': model.fc2.weight.grad.abs().var(),
            'fc3': model.fc3.weight.grad.abs().var(),
            'fc4': model.fc4.weight.grad.abs().var()
            
        }, epoch)
        
        tb.add_scalars('fc_grad/max',{
            'fc1': model.fc1.weight.grad.abs().max(),
            'fc2': model.fc2.weight.grad.abs().max(),
            'fc3': model.fc3.weight.grad.abs().max(),
            'fc4': model.fc4.weight.grad.abs().max()
            
        }, epoch)
        
        optimizer.step()
        losses.append(loss.item())
    
    mean_loss = sum(losses)/len(losses)
    return mean_loss
        
def evaluate():
    model.eval()
    with torch.no_grad():
        losses = []
        for j, (data, label) in enumerate(val_dataloader, 0):
            data = data.to(device).view(data.shape[0], data.shape[2], data.shape[1])
            y = label.to(device)
            output = model(data)
            losses.append(criterion(output, y).item())
    return sum(losses)/len(losses)

In [21]:
num_epoch = 300
train_loss = []
val_loss = []
best_val_loss = np.inf

for i, epoch in enumerate(range(num_epoch)):
    train_loss.append(train(epoch))
    val_loss.append(evaluate())
    
    tb.add_scalars('Loss', {'Train loss': train_loss[i], 'Val loss': val_loss[i]}, epoch)
    tb.add_scalars('fc_weight/abs_mean',{
            'fc1': model.fc1.weight.abs().mean(),
            'fc2': model.fc2.weight.abs().mean(),
            'fc3': model.fc3.weight.abs().mean(),
            'fc4': model.fc4.weight.abs().mean()
        }, epoch)
    
    tb.add_scalars('fc_weight/abd_var',{
            'fc1': model.fc1.weight.abs().var(),
            'fc2': model.fc2.weight.abs().var(),
            'fc3': model.fc3.weight.abs().var(),
            'fc4': model.fc4.weight.abs().var()
        }, epoch)
        
    tb.add_histogram('weight/fc1', model.fc1.weight, epoch)
    tb.add_histogram('weight/fc2', model.fc2.weight, epoch)
    tb.add_histogram('weight/fc3', model.fc3.weight, epoch)
    tb.add_histogram('weight/fc4', model.fc4.weight, epoch)
    print(f'| Epoch: {epoch}/{num_epoch} | Train loss: {train_loss[i]} | Val loss: {val_loss[i]}')
        
    if val_loss[i] < best_val_loss:
        best_val_loss = val_loss[i]
        best_model = model
        
        best_model_weights = copy.deepcopy(model.state_dict())
        torch.save(best_model_weights, f'./best_models/seismic_net_epoch_{epoch}_val_loss_{val_loss[i]}.pt')

    scheduler.step(val_loss[i])

tb.close()

| Epoch: 0/300 | Train loss: 1.0029791399052268 | Val loss: 0.9932259801361296
| Epoch: 1/300 | Train loss: 0.9994859699617352 | Val loss: 0.9951163712475035
| Epoch: 2/300 | Train loss: 1.0048325480076306 | Val loss: 0.9920800377925237
| Epoch: 3/300 | Train loss: 0.9996001885648359 | Val loss: 0.9892509596215354
| Epoch: 4/300 | Train loss: 0.9946350043280083 | Val loss: 0.9910081012381448
| Epoch: 5/300 | Train loss: 0.9949361491621587 | Val loss: 0.9870519472493066
| Epoch: 6/300 | Train loss: 0.9871781255069532 | Val loss: 0.9800974047846265
| Epoch: 7/300 | Train loss: 0.9831449949950503 | Val loss: 0.9760463221205605
| Epoch: 8/300 | Train loss: 0.9767379390565972 | Val loss: 0.9696811272038354
| Epoch: 9/300 | Train loss: 0.9714652433730009 | Val loss: 0.96396844006247
| Epoch: 10/300 | Train loss: 0.9668804317189936 | Val loss: 0.9650855039556822
| Epoch: 11/300 | Train loss: 0.9646151944210655 | Val loss: 0.9646155453390546
| Epoch: 12/300 | Train loss: 0.9809925562457035 | V

### Train and validation error

In [23]:
title = f'Minimum validation loss: {np.min(val_loss)}'
p = figure(title = title, plot_width=600, plot_height=250)
x = np.arange(len(train_loss))
p.line(x, train_loss, line_width=2, color = 'blue', legend_label = 'Train RMSE')
p.line(x, val_loss, line_width=2, color = 'red', legend_label = 'Val RMSE')
show(p)